<a href="https://colab.research.google.com/github/Neha13022020/Sentiment-Analysis/blob/main/text_preprocessing_and_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('gdrive/')

Mounted at gdrive/


In [2]:
# Importing libraries

import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
from tqdm import tqdm

from sklearn.feature_extraction.text import CountVectorizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

import os

## Text Preprocessing

In [25]:
df = pd.read_csv('/content/gdrive/MyDrive/Forsk_assignment/balanced_review.csv')
print(df.shape)

(1188000, 3)


In [26]:
print(df.isna().sum())

overall         0
reviewText    939
summary       342
dtype: int64


In [27]:
# dropping null values

df = df.dropna()
print(df.shape)

(1186734, 3)


In [28]:
# converting reviews into 2 classes : 0 ---> Bad, 1 ---> Good 

df = df[df['overall'] != 3]
df['Positivity'] = np.where(df['overall'] > 3, 1, 0 )

print(df['Positivity'].value_counts())
df.shape

0    395683
1    395305
Name: Positivity, dtype: int64


(790988, 4)

In [7]:
# https://gist.github.com/sebleier/554280
# we are removing the words from the stop words list: 'no', 'nor', 'not'
# <br /><br /> ==> after the above steps, we are getting "br br"
# we are including them into stop words list
# instead of <br /> if we have <br/> these tags would have revmoved in the 1st step

stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [8]:
# https://stackoverflow.com/a/47091490/4084039


def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [29]:
preprocessed_reviewText = []
preprocessed_summary = []

for sentance in tqdm(df['reviewText'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_reviewText.append(sentance.strip())


for sentance in tqdm(df['summary'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_summary.append(sentance.strip())

len(preprocessed_summary),len(preprocessed_reviewText)

 20%|█▉        | 154668/790988 [00:40<02:48, 3775.84it/s]/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:273: UserWarning: "b'/'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
 77%|███████▋  | 605140/790988 [02:42<00:54, 3408.04it/s]/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
  6%|▌         | 45474/790988 [00:10<02:50, 4368.34it/s]/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:273: UserWarning: "b'..'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
 22%|██▏       | 176764/790988 [00:46<02:20, 4361.94it/s]/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:273: UserWarning: "b'//'" looks like a 

(790988, 790988)

In [10]:
df['preprocessed_summary']=preprocessed_summary

df['preprocessed_reviewText']=preprocessed_reviewText

df = df.drop(['reviewText','summary'],axis=1)
df.shape

(790988, 3)

In [ ]:
df.to_csv('/content/gdrive/MyDrive/Forsk_assignment/preprocessed_balanced_reviews.csv', index = False)

## Vectorizer

In [38]:
df = pd.read_csv('/content/gdrive/MyDrive/Forsk_assignment/preprocessed_balanced_reviews.csv')

In [11]:
df = df.dropna()
print(df.shape)

(790988, 3)


In [39]:
df.columns

Index(['overall', 'Positivity', 'preprocessed_summary',
       'preprocessed_reviewText'],
      dtype='object')

In [13]:
# avg-w2v for reviews

sentance_review =[]
for sentance in df['preprocessed_reviewText']:
    sentance_review.append(sentance.split())

w2v_model = Word2Vec(sentance_review,min_count=5,size=50, workers=4)
w2v_words = list(w2v_model.wv.vocab)

sent_vectors_r = [] # the avg-w2v for each sentence/review is stored in this list
for sent in tqdm(sentance_review): # for each review/sentence
    sent_vec = np.zeros(50) # as word vectors are of zero length 50, you might need to change this to 300 if you use google's w2v
    cnt_words =0 # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in w2v_words:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
    if cnt_words != 0:
        sent_vec /= cnt_words
    sent_vectors_r.append(sent_vec)

print(len(sent_vectors_r))
print(len(sent_vectors_r[0]))

100%|██████████| 790988/790988 [15:35<00:00, 845.78it/s]

790988
50


In [16]:
w2v_model.save("/content/gdrive/MyDrive/Forsk_assignment/word2vec.model")

In [ ]:
# avg-w2v for summary

sentance_summary =[]
for sentance in df['preprocessed_summary']:
    sentance_summary.append(sentance.split())

w2v_model_2 = Word2Vec(sentance_summary,min_count=5,size=50, workers=4)
w2v_words = list(w2v_model_2.wv.vocab)

sent_vectors_s = [] # the avg-w2v for each sentence/review is stored in this list
for sent in tqdm(sentance_summary): # for each review/sentence
    sent_vec = np.zeros(50) # as word vectors are of zero length 50, you might need to change this to 300 if you use google's w2v
    cnt_words =0 # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in w2v_words:
            vec = w2v_model_2.wv[word]
            sent_vec += vec
            cnt_words += 1
    if cnt_words != 0:
        sent_vec /= cnt_words
    sent_vectors_s.append(sent_vec)

print(len(sent_vectors_s))
print(len(sent_vectors_s[0]))

100%|██████████| 789108/789108 [01:13<00:00, 10723.73it/s]

789108
50


In [ ]:
sent_vectors_r = pd.DataFrame(sent_vectors_r)
sent_vectors_s = pd.DataFrame(sent_vectors_s)
print(sent_vectors_r.shape,sent_vectors_s.shape)

new_df = pd.concat([sent_vectors_r,sent_vectors_s], axis=1, ignore_index = True)
new_df.shape

(789108, 50) (789108, 50)


(789108, 100)

In [ ]:
t = []
for i in df['Positivity']:
  t.append(i)
new_df['Target'] = t
new_df.shape

(789108, 101)

In [ ]:
new_df.to_csv('/content/gdrive/MyDrive/Forsk_assignment/vectorized_balanced_reviews.csv', index = False)

## Modeling

In [40]:
new_df = pd.read_csv('/content/gdrive/MyDrive/Forsk_assignment/vectorized_balanced_reviews.csv')

In [41]:
from sklearn.model_selection import train_test_split

Target = new_df['Target']
new_df = new_df.drop(['Target'],axis=1)

features_train, features_test, labels_train, labels_test = train_test_split(new_df, Target, random_state = 42) 
features_train.shape, features_test.shape, labels_train.shape, labels_test.shape

((591831, 100), (197277, 100), (591831,), (197277,))

In [42]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(features_train, labels_train)


perdictions = model.predict(features_test)

from sklearn.metrics import confusion_matrix

print(confusion_matrix(labels_test, perdictions))

from sklearn.metrics import roc_auc_score
roc_auc_score(labels_test, perdictions)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[90135  8346]
 [ 9766 89030]]


0.9082012662295326

In [23]:
# saving the model

import pickle

pkl_filename = "/content/gdrive/MyDrive/Forsk_assignment/pickle_model.pkl"

with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)
